In [1]:
import pandas
import numpy
from scipy.stats import zscore
import seaborn

df = pandas.read_csv('../ThiTHPT2018.csv')
df = df.dropna(axis=1, how='all')
df

C:\Users\Duy Long\AppData\Local\Temp\ipykernel_11864\2955805734.py:6: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv('../ThiTHPT2018.csv')


,SoBD,Toan,Van,Anh,Ly,Hoa,Sinh,Su,Dia,GDCD,Unnamed: 10,KhoiA,KhoiB,KhoiC,KhoiD,KhoiA1,Ma Tinh,Ten Tinh,Vung mien
0,1,3.0,3.75,3.0,NaN,NaN,NaN,3.0,6.50,8.25,NaN,NaN,NaN,13.25,9.75,NaN,18,Bac Giang,Mien Bac
1,2,8.8,7.50,9.0,NaN,NaN,NaN,6.0,9.00,8.25,NaN,NaN,NaN,22.50,25.3,NaN,18,Bac Giang,Mien Bac
2,3,6.0,5.50,4.0,5.75,5.50,5.00,NaN,NaN,NaN,NaN,17.25,16.50,NaN,15.5,15.75,18,Bac Giang,Mien Bac
3,4,3.4,5.75,2.6,NaN,NaN,NaN,3.5,4.75,7.25,NaN,NaN,NaN,14.00,11.75,NaN,18,Bac Giang,Mien Bac
4,5,3.8,6.75,3.0,NaN,NaN,NaN,3.5,6.25,8.00,NaN,NaN,NaN,16.50,13.55,NaN,18,Bac Giang,Mien Bac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744391,744392,4.6,4.50,6.0,3.75,3.00,3.25,NaN,NaN,NaN,NaN,11.35,10.85,NaN,15.1,14.35,52,Ba Ria Vung Tau,Mien Nam
744392,744393,7.0,6.00,5.0,5.25,5.50,4.00,NaN,NaN,NaN,NaN,17.75,16.50,NaN,18,17.25,52,Ba Ria Vung Tau,Mien Nam
744393,744394,5.2,3.50,4.0,6.25,5.50,2.25,NaN,NaN,NaN,NaN,16.95,12.95,NaN,12.7,15.45,52,Ba Ria Vung Tau,Mien Nam
744394,744395,7.8,4.50,5.4,7.50,6.25,3.00,NaN,NaN,NaN,NaN,21.55,17.05,NaN,17.7,20.70,52,Ba Ria Vung Tau,Mien Nam


In [41]:
# nhiều giá trị trong cột KhoiD có kiểu dữ liệu không đúng
invalid_values = df.loc[df['KhoiD'].apply(lambda x: type(x) != float), 'KhoiD']
print(invalid_values)

262144    15.15
262145     17.8
262146     20.2
262147    17.85
262148    16.15
          ...  
744391     15.1
744392       18
744393     12.7
744394     17.7
744395     12.3
Name: KhoiD, Length: 429425, dtype: object


In [3]:
df.isnull().sum()

SoBD              0
Toan           3372
Van           16087
Anh           84398
Ly           421043
Hoa          417415
Sinh         424791
Su           298278
Dia          311175
GDCD         365362
KhoiA        423164
KhoiB        424880
KhoiC        311791
KhoiD         88043
KhoiA1       436532
Ma Tinh           0
Ten Tinh          0
Vung mien         0
dtype: int64

In [4]:
df2 = df.copy()

In [43]:
# thay các record bị thiếu của trường Toán bằng giá trị trung bình
df2['Toan'].fillna(df2['Toan'].mean().round(2), inplace=True)
df2['Toan'].isnull().sum()

0

In [6]:
# thay các record bị thiếu của trường Văn bằng giá trị thường xuyên
df2['Van'].fillna(df2['Van'].mode()[0], inplace=True)
df2['Van'].isnull().sum()

0

In [25]:
# Thay giá trị rỗng của trường Anh bằng giá trị thường xuyên gộp nhóm theo Ma tinh

# Lọc ra giá trị thường xuyên của trường Anh gọp nhóm theo trường Ma Tinh
mode_by_Anh = df2.groupby('Ma Tinh')['Anh'].apply(lambda x: x.mode()[0]).to_dict()
# mode_by_Anh

# Sử dụng hàm apply kết hợp với hàm lambda để thay thế giá trị rỗng trong cột Anh bằng giá trị thường xuyên của cột Anh trong cùng nhóm của cột Ma Tinh. 
# Hàm apply sẽ thực hiện cho từng hàng của dataframe, 
# nếu giá trị của cột Anh là rỗng (sử dụng hàm pd.isnull), thì giá trị mới được thay thế bằng giá trị thường xuyên của cột Anh trong nhóm của cột Ma Tinh đó (mode_by_Anh). 
# Nếu giá trị cột Anh không rỗng thì giữ nguyên giá trị đó.
df2['Anh'] = df2.apply(lambda x: mode_by_Anh[x['Ma Tinh']] if pandas.isnull(x['Anh']) else x['Anh'], axis=1)

In [30]:
df2['Anh'].isna().sum()

0

In [9]:
# Thay thế các giá trị rỗng bằng 'Khong thi'
df_fill_by_khongthi = df.fillna('Khong thi')
df_fill_by_khongthi

,SoBD,Toan,Van,Anh,Ly,Hoa,Sinh,Su,Dia,GDCD,KhoiA,KhoiB,KhoiC,KhoiD,KhoiA1,Ma Tinh,Ten Tinh,Vung mien
0,1,3.0,3.75,3.0,Khong thi,Khong thi,Khong thi,3.0,6.5,8.25,Khong thi,Khong thi,13.25,9.75,Khong thi,18,Bac Giang,Mien Bac
1,2,8.8,7.5,9.0,Khong thi,Khong thi,Khong thi,6.0,9.0,8.25,Khong thi,Khong thi,22.5,25.3,Khong thi,18,Bac Giang,Mien Bac
2,3,6.0,5.5,4.0,5.75,5.5,5.0,Khong thi,Khong thi,Khong thi,17.25,16.5,Khong thi,15.5,15.75,18,Bac Giang,Mien Bac
3,4,3.4,5.75,2.6,Khong thi,Khong thi,Khong thi,3.5,4.75,7.25,Khong thi,Khong thi,14.0,11.75,Khong thi,18,Bac Giang,Mien Bac
4,5,3.8,6.75,3.0,Khong thi,Khong thi,Khong thi,3.5,6.25,8.0,Khong thi,Khong thi,16.5,13.55,Khong thi,18,Bac Giang,Mien Bac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744391,744392,4.6,4.5,6.0,3.75,3.0,3.25,Khong thi,Khong thi,Khong thi,11.35,10.85,Khong thi,15.1,14.35,52,Ba Ria Vung Tau,Mien Nam
744392,744393,7.0,6.0,5.0,5.25,5.5,4.0,Khong thi,Khong thi,Khong thi,17.75,16.5,Khong thi,18,17.25,52,Ba Ria Vung Tau,Mien Nam
744393,744394,5.2,3.5,4.0,6.25,5.5,2.25,Khong thi,Khong thi,Khong thi,16.95,12.95,Khong thi,12.7,15.45,52,Ba Ria Vung Tau,Mien Nam
744394,744395,7.8,4.5,5.4,7.5,6.25,3.0,Khong thi,Khong thi,Khong thi,21.55,17.05,Khong thi,17.7,20.7,52,Ba Ria Vung Tau,Mien Nam


In [10]:
# Xóa những cột có giá trị thiếu >= 50%
df_delete = df.copy()
half_df = len(df_delete)/2
df_delete = df_delete.dropna(thresh=half_df, axis=1)
df_delete

,SoBD,Toan,Van,Anh,Su,Dia,GDCD,KhoiC,KhoiD,Ma Tinh,Ten Tinh,Vung mien
0,1,3.0,3.75,3.0,3.0,6.50,8.25,13.25,9.75,18,Bac Giang,Mien Bac
1,2,8.8,7.50,9.0,6.0,9.00,8.25,22.50,25.3,18,Bac Giang,Mien Bac
2,3,6.0,5.50,4.0,NaN,NaN,NaN,NaN,15.5,18,Bac Giang,Mien Bac
3,4,3.4,5.75,2.6,3.5,4.75,7.25,14.00,11.75,18,Bac Giang,Mien Bac
4,5,3.8,6.75,3.0,3.5,6.25,8.00,16.50,13.55,18,Bac Giang,Mien Bac
...,...,...,...,...,...,...,...,...,...,...,...,...
744391,744392,4.6,4.50,6.0,NaN,NaN,NaN,NaN,15.1,52,Ba Ria Vung Tau,Mien Nam
744392,744393,7.0,6.00,5.0,NaN,NaN,NaN,NaN,18,52,Ba Ria Vung Tau,Mien Nam
744393,744394,5.2,3.50,4.0,NaN,NaN,NaN,NaN,12.7,52,Ba Ria Vung Tau,Mien Nam
744394,744395,7.8,4.50,5.4,NaN,NaN,NaN,NaN,17.7,52,Ba Ria Vung Tau,Mien Nam


In [11]:
# Điền giá trị còn thiếu bằng 0
df_fill_by_0 = df.fillna(0)
df_fill_by_0

,SoBD,Toan,Van,Anh,Ly,Hoa,Sinh,Su,Dia,GDCD,KhoiA,KhoiB,KhoiC,KhoiD,KhoiA1,Ma Tinh,Ten Tinh,Vung mien
0,1,3.0,3.75,3.0,0.00,0.00,0.00,3.0,6.50,8.25,0.00,0.00,13.25,9.75,0.00,18,Bac Giang,Mien Bac
1,2,8.8,7.50,9.0,0.00,0.00,0.00,6.0,9.00,8.25,0.00,0.00,22.50,25.3,0.00,18,Bac Giang,Mien Bac
2,3,6.0,5.50,4.0,5.75,5.50,5.00,0.0,0.00,0.00,17.25,16.50,0.00,15.5,15.75,18,Bac Giang,Mien Bac
3,4,3.4,5.75,2.6,0.00,0.00,0.00,3.5,4.75,7.25,0.00,0.00,14.00,11.75,0.00,18,Bac Giang,Mien Bac
4,5,3.8,6.75,3.0,0.00,0.00,0.00,3.5,6.25,8.00,0.00,0.00,16.50,13.55,0.00,18,Bac Giang,Mien Bac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744391,744392,4.6,4.50,6.0,3.75,3.00,3.25,0.0,0.00,0.00,11.35,10.85,0.00,15.1,14.35,52,Ba Ria Vung Tau,Mien Nam
744392,744393,7.0,6.00,5.0,5.25,5.50,4.00,0.0,0.00,0.00,17.75,16.50,0.00,18,17.25,52,Ba Ria Vung Tau,Mien Nam
744393,744394,5.2,3.50,4.0,6.25,5.50,2.25,0.0,0.00,0.00,16.95,12.95,0.00,12.7,15.45,52,Ba Ria Vung Tau,Mien Nam
744394,744395,7.8,4.50,5.4,7.50,6.25,3.00,0.0,0.00,0.00,21.55,17.05,0.00,17.7,20.70,52,Ba Ria Vung Tau,Mien Nam


In [12]:
# Điền giá trị còn thiếu bằng NaN
df_fill_by_NaN = df.fillna('NaN')
df_fill_by_NaN

,SoBD,Toan,Van,Anh,Ly,Hoa,Sinh,Su,Dia,GDCD,KhoiA,KhoiB,KhoiC,KhoiD,KhoiA1,Ma Tinh,Ten Tinh,Vung mien
0,1,3.0,3.75,3.0,NaN,NaN,NaN,3.0,6.5,8.25,NaN,NaN,13.25,9.75,NaN,18,Bac Giang,Mien Bac
1,2,8.8,7.5,9.0,NaN,NaN,NaN,6.0,9.0,8.25,NaN,NaN,22.5,25.3,NaN,18,Bac Giang,Mien Bac
2,3,6.0,5.5,4.0,5.75,5.5,5.0,NaN,NaN,NaN,17.25,16.5,NaN,15.5,15.75,18,Bac Giang,Mien Bac
3,4,3.4,5.75,2.6,NaN,NaN,NaN,3.5,4.75,7.25,NaN,NaN,14.0,11.75,NaN,18,Bac Giang,Mien Bac
4,5,3.8,6.75,3.0,NaN,NaN,NaN,3.5,6.25,8.0,NaN,NaN,16.5,13.55,NaN,18,Bac Giang,Mien Bac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744391,744392,4.6,4.5,6.0,3.75,3.0,3.25,NaN,NaN,NaN,11.35,10.85,NaN,15.1,14.35,52,Ba Ria Vung Tau,Mien Nam
744392,744393,7.0,6.0,5.0,5.25,5.5,4.0,NaN,NaN,NaN,17.75,16.5,NaN,18,17.25,52,Ba Ria Vung Tau,Mien Nam
744393,744394,5.2,3.5,4.0,6.25,5.5,2.25,NaN,NaN,NaN,16.95,12.95,NaN,12.7,15.45,52,Ba Ria Vung Tau,Mien Nam
744394,744395,7.8,4.5,5.4,7.5,6.25,3.0,NaN,NaN,NaN,21.55,17.05,NaN,17.7,20.7,52,Ba Ria Vung Tau,Mien Nam


In [13]:
# Điền giá trị trung bình cho những cột có giá trị thiếu < 20%
df2 = df.copy()

for i in df2:
   rate = (df2[i].isnull().sum() / df2[i].shape[0]) * 100
   is_null_sum = df2[i].isnull().sum()
   is_number = df[i].dtype in [pandas.Int32Dtype(), pandas.Int64Dtype(), 'int32', 'int64', 'float32', 'float64']
   # kiểm tra điều kiện trước khi điền giá trị
   # Số phần tử bị rỗng trong cột đó (rate) < 20%
   # Phải có ít nhất 1 phần tử rỗng trong cột (is_null_sum)
   # Kiểu dữ liệu các phần tử trong cột đó phải là kiểu số mới tính đc giá trị trung bình 
   if rate < 20 and is_null_sum > 0 and is_number:
      df2[i].fillna(df2[i].mean())

In [29]:
df2

,SoBD,Toan,Van,Anh,Ly,Hoa,Sinh,Su,Dia,GDCD,KhoiA,KhoiB,KhoiC,KhoiD,KhoiA1,Ma Tinh,Ten Tinh,Vung mien
0,1,3.0,3.75,3.0,-8.801604e-16,8.794254e-16,NaN,3.0,1.059669e+00,8.25,NaN,NaN,13.25,9.75,NaN,18,Bac Giang,Mien Bac
1,2,8.8,7.50,9.0,-8.801604e-16,8.794254e-16,NaN,6.0,3.665826e+00,8.25,NaN,NaN,22.50,25.3,NaN,18,Bac Giang,Mien Bac
2,3,6.0,5.50,4.0,7.817003e-01,6.293447e-01,5.00,NaN,-9.258930e-16,NaN,17.25,16.50,NaN,15.5,15.75,18,Bac Giang,Mien Bac
3,4,3.4,5.75,2.6,-8.801604e-16,8.794254e-16,NaN,3.5,-7.646408e-01,7.25,NaN,NaN,14.00,11.75,NaN,18,Bac Giang,Mien Bac
4,5,3.8,6.75,3.0,-8.801604e-16,8.794254e-16,NaN,3.5,7.990534e-01,8.00,NaN,NaN,16.50,13.55,NaN,18,Bac Giang,Mien Bac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744391,744392,4.6,4.50,6.0,-1.200245e+00,-1.846018e+00,3.25,NaN,-9.258930e-16,NaN,11.35,10.85,NaN,15.1,14.35,52,Ba Ria Vung Tau,Mien Nam
744392,744393,7.0,6.00,5.0,2.862140e-01,6.293447e-01,4.00,NaN,-9.258930e-16,NaN,17.75,16.50,NaN,18,17.25,52,Ba Ria Vung Tau,Mien Nam
744393,744394,5.2,3.50,4.0,1.277186e+00,6.293447e-01,2.25,NaN,-9.258930e-16,NaN,16.95,12.95,NaN,12.7,15.45,52,Ba Ria Vung Tau,Mien Nam
744394,744395,7.8,4.50,5.4,2.515902e+00,1.371953e+00,3.00,NaN,-9.258930e-16,NaN,21.55,17.05,NaN,17.7,20.70,52,Ba Ria Vung Tau,Mien Nam


In [15]:
# Thay các giá trị rỗng trong các cột bằng giá trị trung bình
cols_to_fillna = ['Hoa', 'Ly', 'Dia']
for col in cols_to_fillna:
   df2[col].fillna(df2[col].mean(), inplace=True)

In [16]:
df2[cols_to_fillna].isnull().sum()

Hoa    0
Ly     0
Dia    0
dtype: int64

In [17]:

df2[['Hoa','Ly','Dia']] = zscore(df2[['Hoa','Ly','Dia']])

In [18]:
df2

,SoBD,Toan,Van,Anh,Ly,Hoa,Sinh,Su,Dia,GDCD,KhoiA,KhoiB,KhoiC,KhoiD,KhoiA1,Ma Tinh,Ten Tinh,Vung mien
0,1,3.0,3.75,3.0,-8.801604e-16,8.794254e-16,NaN,3.0,1.059669e+00,8.25,NaN,NaN,13.25,9.75,NaN,18,Bac Giang,Mien Bac
1,2,8.8,7.50,9.0,-8.801604e-16,8.794254e-16,NaN,6.0,3.665826e+00,8.25,NaN,NaN,22.50,25.3,NaN,18,Bac Giang,Mien Bac
2,3,6.0,5.50,4.0,7.817003e-01,6.293447e-01,5.00,NaN,-9.258930e-16,NaN,17.25,16.50,NaN,15.5,15.75,18,Bac Giang,Mien Bac
3,4,3.4,5.75,2.6,-8.801604e-16,8.794254e-16,NaN,3.5,-7.646408e-01,7.25,NaN,NaN,14.00,11.75,NaN,18,Bac Giang,Mien Bac
4,5,3.8,6.75,3.0,-8.801604e-16,8.794254e-16,NaN,3.5,7.990534e-01,8.00,NaN,NaN,16.50,13.55,NaN,18,Bac Giang,Mien Bac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744391,744392,4.6,4.50,6.0,-1.200245e+00,-1.846018e+00,3.25,NaN,-9.258930e-16,NaN,11.35,10.85,NaN,15.1,14.35,52,Ba Ria Vung Tau,Mien Nam
744392,744393,7.0,6.00,5.0,2.862140e-01,6.293447e-01,4.00,NaN,-9.258930e-16,NaN,17.75,16.50,NaN,18,17.25,52,Ba Ria Vung Tau,Mien Nam
744393,744394,5.2,3.50,4.0,1.277186e+00,6.293447e-01,2.25,NaN,-9.258930e-16,NaN,16.95,12.95,NaN,12.7,15.45,52,Ba Ria Vung Tau,Mien Nam
744394,744395,7.8,4.50,5.4,2.515902e+00,1.371953e+00,3.00,NaN,-9.258930e-16,NaN,21.55,17.05,NaN,17.7,20.70,52,Ba Ria Vung Tau,Mien Nam


In [47]:
df2.to_csv('TotnghiepOK.csv')